In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import Sequential
#!pip install tensorflow keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

print(tf.__version__)


2.18.0


In [2]:
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112475 entries, 0 to 112474
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   dteday        112475 non-null  object 
 1   hr            112475 non-null  float64
 2   casual        112475 non-null  int64  
 3   registered    112475 non-null  int64  
 4   temp_c        112475 non-null  float64
 5   feels_like_c  112475 non-null  float64
 6   hum           112475 non-null  float64
 7   windspeed     112475 non-null  float64
 8   weathersit    112475 non-null  int64  
 9   season        112475 non-null  int64  
 10  holiday       112475 non-null  int64  
 11  workingday    112475 non-null  int64  
dtypes: float64(5), int64(6), object(1)
memory usage: 10.3+ MB


In [3]:
bikes.head()

,dteday,hr,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday
0,1/1/2011,0.0,3,13,3.0,3.0,0.7957,0.8,1,1,0,0
1,1/1/2011,1.0,8,30,1.7,1.7,0.8272,0.8,1,1,0,0
2,1/1/2011,2.0,5,26,1.9,1.9,0.8157,1.1,1,1,0,0
3,1/1/2011,3.0,3,9,2.5,2.5,0.7831,0.8,1,1,0,0
4,1/1/2011,4.0,0,1,2.0,2.0,0.8075,1.1,1,1,0,0


In [4]:
#drop NA values
bikes.dropna(axis=0, how="any", inplace=True)

In [5]:
#get the dteday into the right format to use for tensorflow
bikes_date_str = bikes['dteday'].astype(str)  #convert to a string
bikes_date = pd.to_datetime(bikes_date_str)   #change string to datetime
bikes["dteday"] = bikes_date  #add to frame
bikes["dteday"] = bikes["dteday"] + pd.to_timedelta(bikes["hr"], unit='h')  #add the hours as a datetime timedelta
bikes.tail()

,dteday,hr,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday
112470,2023-10-31 19:00:00,19.0,241,594,10.2,10.2,0.4516,8.4,2,4,0,1
112471,2023-10-31 20:00:00,20.0,171,450,8.9,7.5,0.5268,9.3,1,4,0,1
112472,2023-10-31 21:00:00,21.0,176,402,7.7,6.4,0.5756,7.8,1,4,0,1
112473,2023-10-31 22:00:00,22.0,106,257,8.0,8.0,0.5604,0.0,1,4,0,1
112474,2023-10-31 23:00:00,23.0,78,135,6.5,6.5,0.6401,4.5,1,4,0,1


In [6]:
#convert datetime into epochs
bikes['dteday'] = bikes['dteday'].astype(int) / 10**9 #convert to seconds

#remove hour column
bikes = bikes.drop(columns=['hr'])

bikes.head()

,dteday,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday
0,1.293840e+09,3,13,3.0,3.0,0.7957,0.8,1,1,0,0
1,1.293844e+09,8,30,1.7,1.7,0.8272,0.8,1,1,0,0
2,1.293847e+09,5,26,1.9,1.9,0.8157,1.1,1,1,0,0
3,1.293851e+09,3,9,2.5,2.5,0.7831,0.8,1,1,0,0
4,1.293854e+09,0,1,2.0,2.0,0.8075,1.1,1,1,0,0


In [7]:
#create two dataframes one for predicting registered and one for predicting casual
bikes_reg = bikes.drop(columns=['casual'])
bikes_casual = bikes.drop(columns=['registered'])
bikes_casual.head()

,dteday,casual,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday
0,1.293840e+09,3,3.0,3.0,0.7957,0.8,1,1,0,0
1,1.293844e+09,8,1.7,1.7,0.8272,0.8,1,1,0,0
2,1.293847e+09,5,1.9,1.9,0.8157,1.1,1,1,0,0
3,1.293851e+09,3,2.5,2.5,0.7831,0.8,1,1,0,0
4,1.293854e+09,0,2.0,2.0,0.8075,1.1,1,1,0,0


In [8]:
#train test split
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(bikes_reg, bikes["casual"], test_size=0.3, random_state=42)

#scale the data
scaler = MinMaxScaler().fit(x_test)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)



In [9]:
#build a sequential NN
model = Sequential()
model.add(layers.Dense(1, input_dim=len(x_train[0]), activation='relu'))
model.add(layers.Dense(1, activation='relu'))

model.summary()

#complie module
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=0.001), metrics=['mean_squared_error'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 1)                   │              11 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               2 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13 (52.00 B)

 Trainable params: 13 (52.00 B)

 Non-trainable params: 0 (0.00 B)